# Analysing Data Scientist Salaries

In this notebook, we will analyze the [Kaggle Jobs Dataset from Glassdoor](https://www.kaggle.com/datasets/thedevastator/jobs-dataset-from-glassdoor) that contains job postings from Glassdoor.com from 2017. 

We aim to analyze the dataset considering the following research questions:

- What are the key factors that affect data science salaries? 
- Can we predict the salary of data science positions based on the job postings?  

## Setup the dataset to HDFS for big data analysis

The HDFS will allow us to store and retrieve the data efficiently for our analysis. It makes sure the data is readily available in parallel.

Before loading the data to HDFS we added the missing header **ID** into files `eda_data.csv` and `glassdoor_jobs.csv`.

In [1]:
# TODO: SET UP COMMANDS

## EDA

In [2]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark import SparkContext, SparkConf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()


conf = SparkConf().set('spark.ui.port', '4040')
sc = SparkContext(conf=conf)
spark = SparkSession.builder.appName('Data Scientist Salaries').master('local[*]').getOrCreate()

24/10/23 11:59:48 WARN Utils: Your hostname, MacBook-Pro-Eric.local resolves to a loopback address: 127.0.0.1; using 213.112.119.200 instead (on interface en0)
24/10/23 11:59:48 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/23 11:59:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark

In [4]:
# read the dataset, ecape quotes inside the job descriptions
df = spark.read.csv('./dataset/eda_data.csv', header=True, inferSchema=True, multiLine=True, quote='"', escape='"', mode='PERMISSIVE')
df.show(5)

24/10/23 11:59:55 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+---+--------------------+--------------------+--------------------+------+--------------------+---------------+--------------+--------------------+-------+------------------+--------------------+--------------------+--------------------+--------------------+------+-----------------+----------+----------+----------+--------------------+---------+----------+---+---------+----+-----+---+-----+--------------+---------+--------+--------+
| ID|           Job Title|     Salary Estimate|     Job Description|Rating|        Company Name|       Location|  Headquarters|                Size|Founded| Type of ownership|            Industry|              Sector|             Revenue|         Competitors|hourly|employer_provided|min_salary|max_salary|avg_salary|         company_txt|job_state|same_state|age|python_yn|R_yn|spark|aws|excel|      job_simp|seniority|desc_len|num_comp|
+---+--------------------+--------------------+--------------------+------+--------------------+---------------+------------

In [5]:
print('Number of datapoints:', df.count())
print('Number of columns:', len(df.columns))

Number of datapoints: 742
Number of columns: 33


**Show schema:**

In [6]:
df.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- Job Title: string (nullable = true)
 |-- Salary Estimate: string (nullable = true)
 |-- Job Description: string (nullable = true)
 |-- Rating: double (nullable = true)
 |-- Company Name: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Headquarters: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Founded: integer (nullable = true)
 |-- Type of ownership: string (nullable = true)
 |-- Industry: string (nullable = true)
 |-- Sector: string (nullable = true)
 |-- Revenue: string (nullable = true)
 |-- Competitors: string (nullable = true)
 |-- hourly: integer (nullable = true)
 |-- employer_provided: integer (nullable = true)
 |-- min_salary: integer (nullable = true)
 |-- max_salary: integer (nullable = true)
 |-- avg_salary: double (nullable = true)
 |-- company_txt: string (nullable = true)
 |-- job_state: string (nullable = true)
 |-- same_state: integer (nullable = true)
 |-- age: integer (nullable = 

### Data Cleaning

First, we will fix our dataset with regard to missing values and either fill the instances or drop the records containing these values.

**Handling of irrelevant features:**

The dataset contains a lot of columns that are interesting and could be used for the predicitons.  We have decided to drop features that will not be necessary for our analysis, from the data set, to decrease dimensionality and also the number of missing values that need to be filled. Since the `Job Description` column contains a long string of text we choose to drop and use `desc_len` as an inidicator of how detailed the descriptions for each job are. We also do not care about where a single person wants to work or their feature so we drop columns `same_state` and `age`. `Company Name`is just an unclean version of `company_txt`, and `Salary Estimate` is already diveded to `min_salary`, `avg_salary` and `max_salary`.

In [7]:
def drop_irrelevant_cols(df):
    df = df.drop(*['Job Description', 'age', 'same_state', 'Competitors', 'Company Name', 'Salary Estimate'])
    return df

df = drop_irrelevant_cols(df)

In [8]:
df.describe().show()

+-------+------------------+-----------------+------------------+----------------+-------------------+-------+-----------------+-----------------+---------+----------------+--------------------+-------------------+--------------------+-----------------+------------------+------------------+--------------------+---------+-------------------+--------------------+-------------------+-------------------+-------------------+--------+---------+------------------+------------------+
|summary|                ID|        Job Title|            Rating|        Location|       Headquarters|   Size|          Founded|Type of ownership| Industry|          Sector|             Revenue|             hourly|   employer_provided|       min_salary|        max_salary|        avg_salary|         company_txt|job_state|          python_yn|                R_yn|              spark|                aws|              excel|job_simp|seniority|          desc_len|          num_comp|
+-------+------------------+----------

**Handling missing values:**

The dataset uses value `-1` to indicate missing values on most columns, which is also visible in the dataset description above showing the summaries per column. We will deal with these column by column case, and fill them with actual data accordingly.

In [9]:
cols_missing_values = []
for col in df.columns:
    missing_count = df.filter(F.col(col) == -1).count()
    if missing_count > 0:
        cols_missing_values.append((col, missing_count))

print("Columns with missing values:", cols_missing_values)

Columns with missing values: [('Rating', 11), ('Headquarters', 1), ('Size', 1), ('Founded', 50), ('Type of ownership', 1), ('Industry', 10), ('Sector', 10), ('Revenue', 1)]


In [10]:
# after playing around with the data by checking the following condition for each column above, 
# it seems that one of the records seem to have many missing values, so we drop it
bad_record = df.filter(df['Headquarters'] == -1)
bad_record.show()

+---+--------------------+------+-------------+------------+----+-------+-----------------+--------+------+-------+------+-----------------+----------+----------+----------+--------------------+---------+---------+----+-----+---+-----+--------+---------+--------+--------+
| ID|           Job Title|Rating|     Location|Headquarters|Size|Founded|Type of ownership|Industry|Sector|Revenue|hourly|employer_provided|min_salary|max_salary|avg_salary|         company_txt|job_state|python_yn|R_yn|spark|aws|excel|job_simp|seniority|desc_len|num_comp|
+---+--------------------+------+-------------+------------+----+-------+-----------------+--------+------+-------+------+-----------------+----------+----------+----------+--------------------+---------+---------+----+-----+---+-----+--------+---------+--------+--------+
|581|Scientist – Cance...|  -1.0|Cambridge, MA|          -1|  -1|     -1|               -1|      -1|    -1|     -1|     0|                1|       100|       135|     117.5|Monte Ro

Drop the corrputed sample:

In [11]:
df = df.filter(df['ID'] != 581)
df.describe().show()

+-------+------------------+-----------------+------------------+----------------+-------------------+-----------------+------------------+--------------------+---------+----------------+--------------------+--------------------+--------------------+------------------+------------------+------------------+--------------------+---------+------------------+--------------------+-------------------+-------------------+------------------+--------+---------+------------------+------------------+
|summary|                ID|        Job Title|            Rating|        Location|       Headquarters|             Size|           Founded|   Type of ownership| Industry|          Sector|             Revenue|              hourly|   employer_provided|        min_salary|        max_salary|        avg_salary|         company_txt|job_state|         python_yn|                R_yn|              spark|                aws|             excel|job_simp|seniority|          desc_len|          num_comp|
+-------+-

Check again after dropping faulty sample:

In [12]:
cols_missing_values = []
for col in df.columns:
    missing_count = df.filter(F.col(col) == -1).count()
    if missing_count > 0:
        cols_missing_values.append((col, missing_count))

print("Columns with missing values:", cols_missing_values)

Columns with missing values: [('Rating', 10), ('Founded', 49), ('Industry', 9), ('Sector', 9)]


The `Rating`is a value in range $[0, 5]$, `Founded` corresponds to a year, `Industry` and `Sector` are categoricals that already have a variable for value *Unknown*. 

For `Rating` we decided that assuming the mean is reasonable for missing values, and for `Founded` the median (most common year). For `Industry` and `Sector` we can use the already defined *Unknown* value to fill the `-1`s.

In [13]:
# Find the median, max and min of the founded column
min_max_median = df.filter(df['Founded'] != -1).agg(
    F.min('Founded').alias('min_value'),
    F.max('Founded').alias('max_value'),
    F.expr('percentile_approx(Founded, 0.5)').alias('median_value')  # 0.5 for median
)

min_max_median.show()

+---------+---------+------------+
|min_value|max_value|median_value|
+---------+---------+------------+
|     1744|     2019|        1992|
+---------+---------+------------+



With further exploration we found missing values (`-1`or `na`) per column accordingly:

- Rating - 11  -> `3` (mean)
- Industry - `9`
- Sector - 9 -> `Unknown`
- Founded - 49 -> `1992` (median)
- seniority - 519 -> `med` (medium experience)
- job_simp - 183 -> `vague` (not a specific datascience role but more vague such as research scientist)


The arrow points out the value to be used as the filler.

In [14]:
def fill_missing_values(df):
    df = df.withColumn('Rating', F.when(df['Rating'] == -1, 3.0).otherwise(df['Rating']))  # use the mean
    df = df.withColumn('Industry', F.when(df['Industry'] == -1, 'Unknown').otherwise(df['Industry']))  # the column already has a label Unknown so we use it
    df = df.withColumn('Founded', F.when(df['Founded'] == -1, 1992).otherwise(df['Founded']))  # fill in median year founded

    # change the na of seniotrity to mean mediocare experience requirements
    df = df.withColumn('seniority', F.when(df['seniority'] == 'na', 'med').otherwise(df['seniority'])) 
    # not a clearly deined job position, research or scientist positons, devops/spark engineers or many tasks etc
    df = df.withColumn('job_simp', F.when(df['job_simp'] == 'na', 'vague').otherwise(df['job_simp'])) 
    return df

df = fill_missing_values(df)

In [15]:
df.groupBy('job_simp').count().show()

+--------------+-----+
|      job_simp|count|
+--------------+-----+
|data scientist|  279|
|         vague|  183|
|      director|   14|
|       manager|   22|
| data engineer|  119|
|       analyst|  102|
|           mle|   22|
+--------------+-----+



In [16]:
df.dtypes

[('ID', 'int'),
 ('Job Title', 'string'),
 ('Rating', 'double'),
 ('Location', 'string'),
 ('Headquarters', 'string'),
 ('Size', 'string'),
 ('Founded', 'int'),
 ('Type of ownership', 'string'),
 ('Industry', 'string'),
 ('Sector', 'string'),
 ('Revenue', 'string'),
 ('hourly', 'int'),
 ('employer_provided', 'int'),
 ('min_salary', 'int'),
 ('max_salary', 'int'),
 ('avg_salary', 'double'),
 ('company_txt', 'string'),
 ('job_state', 'string'),
 ('python_yn', 'int'),
 ('R_yn', 'int'),
 ('spark', 'int'),
 ('aws', 'int'),
 ('excel', 'int'),
 ('job_simp', 'string'),
 ('seniority', 'string'),
 ('desc_len', 'int'),
 ('num_comp', 'int')]

In [17]:
df.describe().show()

+-------+------------------+-----------------+------------------+----------------+-------------------+-----------------+------------------+--------------------+----------+----------------+--------------------+--------------------+--------------------+------------------+------------------+------------------+--------------------+---------+------------------+--------------------+-------------------+-------------------+------------------+--------+---------+------------------+------------------+
|summary|                ID|        Job Title|            Rating|        Location|       Headquarters|             Size|           Founded|   Type of ownership|  Industry|          Sector|             Revenue|              hourly|   employer_provided|        min_salary|        max_salary|        avg_salary|         company_txt|job_state|         python_yn|                R_yn|              spark|                aws|             excel|job_simp|seniority|          desc_len|          num_comp|
+-------

## Visualizations

TODO: find good relation ships for regression models

In [18]:
seed = 42

train_df, test_df = df.randomSplit(weights=[0.8,0.2], seed=seed)
print('Train size:', train_df.count(), '-- Test size:', test_df.count())

Train size: 622 -- Test size: 119
